# Multithreading in Julia

_Part of this notebook is inspired by the material of th [Julia for HPC Course @ UCL ARC ](https://github.com/carstenbauer/JuliaUCL24) by Carsten Bauer._

## Setup

In [1]:
# Running this cell is important to make sure we install all the necessary packages.
using Pkg
Pkg.instantiate()

## Thread pinning

In [2]:
using ThreadPinning
pinthreads(:cores)
threadinfo(; slurm=ThreadPinning.SLURM.isslurmjob())

Hostname: 	springhawk
CPU(s): 	1 x Intel(R) Core(TM) Ultra 7 155H
CPU target: 	alderlake
Cores: 		16 (22 CPU-threads due to 2-way SMT)
NUMA domains: 	1 (16 cores each)

Julia threads: 	22

CPU socket 1
  0,5, 1,2, 3,4, 6,7, 8,9, 10,11, 12, 13, 14, 15, 
  16, 17, 18, 19, 20, 21


# = Julia thread, # = Julia thread on HT, # = >1 Julia thread, # = Efficiency core

(Mapping: 1 => 0, 2 => 1, 3 => 3, 4 => 6, 5 => 8, ...)


## Spawning parallel tasks

In [3]:
using Base.Threads

@show nthreads();

nthreads() = 22


In [4]:
@time t = @spawn begin # `@spawn` returns right away
    sleep(2)
    3+3
end

@time fetch(t) # `fetch` waits for the task to finish

  0.000131 seconds (92 allocations: 4.672 KiB)
  1.974713 seconds (128 allocations: 9.156 KiB)


6

## Exercise: task-based parallelised `map`

In [7]:
using LinearAlgebra, BenchmarkTools

BLAS.set_num_threads(1) # Fix number of BLAS threads

# Exercise: define a task-based `map` function, using tasks
function tmap(fn, itr)
    # Define a variable called `tasks`, which is a vector, which for all element `x` of `itr` holds the task running `fn(x)`
    # Hint #1: you can use `map(f, itr)` to  apply the function `f` on each element of `itr`.
    # Hint #2: anonymous functions `x -> f(x)` are a convenient syntax for defining one-line functions inside other functions.
    tasks = map(i -> @spawn(fn(i)), itr)
    # Call `fetch` on all elements of `tasks` to collect the result of all spawned tasks, and return the result.
    # Hint: you can use broadcasting for running a function element-wise on an iterator.
    return fetch.(tasks)
end

M = [rand(100,100) for i in 1:(8 * nthreads())];

tmap(svdvals, M);

@btime  map(svdvals, $M) samples=10 evals=3;
@btime tmap(svdvals, $M) samples=10 evals=3;

  39.731 ms (1458 allocations: 15.09 MiB)
  5.927 ms (2027 allocations: 15.13 MiB)


***Bonus***: do you see any difference if you increase the number of BLAS threads?

## Exercise: multi-threaded `for` loop (reduction)

In [9]:
using ChunkSplitters, Base.Threads, BenchmarkTools

# Define a function which computes the sum of the elements of an iterator `data` in parallel,
# after applying a user-supplied function `fn` element-wise
function sum_threads(fn, data; nchunks=nthreads())
    psums = zeros(eltype(data), nchunks)
    # Hint: place `@threads` in front of the `for` loop run it multi-threaded.
    # Tip: `ChunkSplitters.chunk` lets you split the data into multiple blocks,
    # which can then be run in parallel.
    @threads for (c, elements) in enumerate(chunks(data; n=nchunks))
        # Hint: each element of `psums` should be the sum of `fn` applied
        # elelementwise to all the items of the current iteration.
        # Tip: read the docstring of `sum`: https://docs.julialang.org/en/v1/base/collections/#Base.sum
        psums[c] = sum(fn, elements)
    end
    return sum(psums)
end

v = randn(20_000_000);

@btime sum(sin, $v);

@btime sum_threads(sin, $v);

  1.436 s (0 allocations: 0 bytes)
  125.114 ms (161 allocations: 10.56 KiB)


***Bonus***: [`Threads.@threads`](https://docs.julialang.org/en/v1/base/multi-threading/#Base.Threads.@threads) lets you choose the scheduler.  Do you see differences if you change the scheduler type?  Remember you can choose between `:dynamic` (currently the default if omitted), `:greedy` (only available when using Julia v1.11+), and `:static`.

In [8]:
# Define a function which does the parallel sum using `map` + `@spawn`
# as we've done above, instead of `@threads for`.
function sum_map_spawn(fn, data; nchunks=nthreads())
    ts = map(chunks(data, n=nchunks)) do elements
        @spawn sum(fn, elements)
    end
    return sum(fetch.(ts))
end

@btime sum_map_spawn(sin, $v);

  17.826 ms (156 allocations: 9.41 KiB)


### Bonus: using OhMyThreads.jl

In [10]:
using OhMyThreads: @tasks

# The package OhMyThreads (https://juliafolds2.github.io/OhMyThreads.jl/) provides user-friendly
# constructs for task-based multithreaded computing.  Define a function which uses
# `OhMyThreads`' `@tasks` instead of `Threads.@threads`.
function sum_tasks(fn, data; nchunks=nthreads())
    psums = zeros(eltype(data), nchunks)
    # Hint: this function will look a lot like `sum_threads` above,
    # but with `@tasks` instead of `@threads`.
    @tasks for (c, elements) in enumerate(chunks(data; n=nchunks))
        psums[c] = sum(fn, elements)
    end
    return sum(psums)
end

@btime sum_tasks(sin, $v);

  18.215 ms (148 allocations: 9.94 KiB)


In [11]:
# `OhMyThreads` provides also a function called `tmapreduce` which does a
# task-based multi-threaded `mapreduce`, pretty much what we wanted to do above.
using OhMyThreads: tmapreduce

@btime tmapreduce(sin, +, $v);

  17.894 ms (190 allocations: 10.69 KiB)


## Multi-threading: is it always worth it?

In [10]:
using BenchmarkTools

function overhead!(v)
    for idx in eachindex(v)
        v[idx] = idx
    end
end
    
function overhead_threads!(v)
    @threads for idx in eachindex(v)
        v[idx] = idx
    end
end

N = 10

@btime overhead!(v) setup=(v = Vector{Int}(undef, N))
@btime overhead_threads!(v) setup=(v = Vector{Int}(undef, N))

  15.661 ns (0 allocations: 0 bytes)
  20.880 μs (81 allocations: 8.17 KiB)


***Exercise***: do you see any improvement in the parallel efficiency if you change the size of the problem (here: `N`)?

## Unbalanced workload: computing hexadecimal $\pi$

_This section is inspired by the blogpost [Computing the hexadecimal value of pi](https://giordano.github.io/blog/2017-11-21-hexadecimal-pi/) by Mosè Giordano._

The [Bailey–Borwein–Plouffe formula](https://en.wikipedia.org/wiki/Bailey%E2%80%93Borwein%E2%80%93Plouffe_formula) is one of the [several algorithms to compute $\pi$](https://en.wikipedia.org/wiki/Approximations_of_%CF%80):

$$
\pi = \sum_{k = 0}^{\infty}\left[ \frac{1}{16^k} \left( \frac{4}{8k + 1} -
\frac{2}{8k + 4} - \frac{1}{8k + 5} - \frac{1}{8k + 6} \right) \right]
$$

What makes this formula stand out among other approximations of $\pi$ is that it allows one to directly extract the $n$-th fractional digit of the hexadecimal value of $\pi$ without computing the preceding ones.

The Wikipedia article about the Bailey–Borwein–Plouffe formula explains that the $n + 1$-th fractional digit $d_n$ is given by

$$
d_{n} = 16 \left[ 4 \Sigma(n, 1) - 2 \Sigma(n, 4) - \Sigma(n, 5) - \Sigma(n,
6) \right]
$$

where

$$
\Sigma(n, j) = \sum_{k = 0}^{n} \frac{16^{n-k} \bmod (8k+j)}{8k+j} + \sum_{k
= n+1}^{\infty} \frac{16^{n-k}}{8k+j}
$$

Only the fractional part of expression in square brackets on the right side of $d_n$ is relevant, thus, in order to avoid rounding errors, when we compute each term of the finite sum above we can take only the fractional part. This allows us to always use ordinary double precision floating-point arithmetic, without resorting to arbitrary-precision numbers. In addition note that the terms of the infinite sum get quickly very small, so we can stop the summation when they become negligible.

### Serial implementation

In [11]:
# Return the fractional part of x, modulo 1, always positive
fpart(x) = mod(x, one(x))

function Σ(n, j)
    # Compute the finite sum
    s = 0.0
    denom = j
    for k in 0:n
        s = fpart(s + powermod(16, n - k, denom) / denom)
        denom += 8
    end
    # Compute the infinite sum
    num = 1 / 16
    while (frac = num / denom) > eps(s)
        s     += frac
        num   /= 16
        denom += 8
    end
    return fpart(s)
end

pi_digit(n) =
    floor(Int, 16 * fpart(4Σ(n-1, 1) - 2Σ(n-1, 4) - Σ(n-1, 5) - Σ(n-1, 6)))

pi_string(n) = "0x3." * join(string.(pi_digit.(1:n), base = 16)) * "p0"

pi_string (generic function with 1 method)

Let's make sure this works:

In [12]:
pi_string(13)

"0x3.243f6a8885a30p0"

In [13]:
# Parse the string as a double-precision floating point number
parse(Float64, pi_string(13))

3.141592653589793

In [14]:
Float64(π) == parse(Float64, pi_string(13))

true

In [15]:
N_pi = 1_000

setprecision(BigFloat, 4 * N_pi) do
    BigFloat(π) == parse(BigFloat, pi_string(N_pi))
end

true

In [16]:
using BenchmarkTools

b = @benchmark pi_string(N_pi)

pi_serial_t = minimum(b.times)

b

BenchmarkTools.Trial: 20 samples with 1 evaluation.
 Range (min … max):  248.124 ms … 264.680 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     249.015 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   251.234 ms ±   4.715 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁█    ▁                                                       
  ███▆▆▆▁█▁▁▁▁▁▁▁▁▆▁▁▁▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  248 ms           Histogram: frequency by time          265 ms <

 Memory estimate: 98.05 KiB, allocs estimate: 2009.

### Multi-threaded implementation

Since the Bailey–Borwtimesn–Plouffe formula extracts the $n$-th digit of $\pi$ without computing the other ones, we can write a multi-threaded version of `pi_string`, taking advantage of native support for [multi-threading](https://docs.julialang.org/en/v1/manual/multi-threading/) in Julia. However note that the computational cost of `pi_digit` is $O(n\log(n))$, so the larger the value of $n$, the longer the function will take, which makes this workload very unbalanced. ***Question***: what do you expect to be the worst performing scheduler?

#### For-loop: static scheduler

In [17]:
function pi_string_threads_static(N)
    digits = Vector{Int}(undef, N)
    @threads :static for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

@assert pi_string_threads_static(N_pi) == pi_string(N_pi)

b = @benchmark pi_string_threads_static(N_pi)

pi_threads_static_t = minimum(b.times)

display(b)

pi_serial_t / pi_threads_static_t / nthreads() * 100

BenchmarkTools.Trial: 139 samples with 1 evaluation.
 Range (min … max):  36.124 ms … 36.867 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     36.169 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.176 ms ± 63.425 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▄ ▂ █▂      ▃▇ ▂                                    
  ▃▃▄▅▆▃▅▇▅███▆██▇▆▅█▆▆████▅▃▃▃▁▄▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  36.1 ms         Histogram: frequency by time        36.3 ms <

 Memory estimate: 114.16 KiB, allocs estimate: 2091.

42.929415179751146

#### For-loop: dynamic scheduler

In [18]:
function pi_string_threads_dynamic(N)
    digits = Vector{Int}(undef, N)
    @threads :dynamic for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

@assert pi_string_threads_dynamic(N_pi) == pi_string(N_pi)

b = @benchmark pi_string_threads_dynamic(N_pi)

pi_threads_dynamic_t = minimum(b.times)

display(b)

pi_serial_t / pi_threads_dynamic_t / nthreads() * 100

BenchmarkTools.Trial: 138 samples with 1 evaluation.
 Range (min … max):  36.132 ms …  37.006 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     36.192 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   36.277 ms ± 175.204 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇▁▇██                                                         
  █████▄▄▃▅▃▃▄▆▄▃▃▃▅▃▄▃▄▁▄▃▃▃▃▄▃▃▃▁▁▁▃▄▁▁▄▃▁▁▁▄▁▁▁▁▁▁▃▁▁▁▁▁▁▁▃ ▃
  36.1 ms         Histogram: frequency by time         36.9 ms <

 Memory estimate: 114.16 KiB, allocs estimate: 2091.

42.9198210415427

#### For-loop: greedy scheduler (only Julia v1.11+)

In [19]:
@static if VERSION >= v"1.11"

function pi_string_threads_greedy(N)
    digits = Vector{Int}(undef, N)
    @threads :greedy for n in eachindex(digits)
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

@assert pi_string_threads_greedy(N_pi) == pi_string(N_pi)
    
b = @benchmark pi_string_threads_greedy(N_pi)

pi_threads_greedy_t = minimum(b.times)

display(b)

pi_serial_t / pi_threads_greedy_t / nthreads() * 100

end

#### Tasks

In [20]:
function pi_string_tasks(N)
    tasks = [Threads.@spawn pi_digit(n) for n in 1:N]
    digits = [fetch(t) for t in tasks]
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

@assert pi_string_tasks(N_pi) == pi_string(N_pi)

b = @benchmark pi_string_tasks(N_pi)

pi_tasks_t = minimum(b.times)

display(b)

pi_serial_t / pi_tasks_t / nthreads() * 100

BenchmarkTools.Trial: 282 samples with 1 evaluation.
 Range (min … max):  17.587 ms …  23.424 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.734 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.761 ms ± 340.401 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                       ▁ ▃▇▇█▇▆▅▆▃▁▁                            
  ▅▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅█████████████▅▅▁▅▅▁▁▅▁▇▁▁▅▅▇▁▅▅▁▁▁▅▅▅▁▅ ▆
  17.6 ms       Histogram: log(frequency) by time      17.9 ms <

 Memory estimate: 598.48 KiB, allocs estimate: 7017.

88.17511123827234

#### Bonus: using OhMyThreads.jl

In [21]:
using OhMyThreads: @tasks

function pi_string_omt(N; ntasks::Int=8 * nthreads(), scheduler::Symbol=:dynamic)
    digits = Vector{Int}(undef, N)
    @tasks for n in eachindex(digits)
        @set ntasks=ntasks
        @set scheduler=scheduler
        digits[n] = pi_digit(n)
    end
    return "0x3." * join(string.(digits, base = 16)) * "p0"
end

@assert pi_string_omt(N_pi) == pi_string(N_pi)

b = @benchmark pi_string_omt(N_pi; ntasks=32 * nthreads())

pi_omt_t = minimum(b.times)

display(b)

pi_serial_t / pi_omt_t / nthreads() * 100

BenchmarkTools.Trial: 282 samples with 1 evaluation.
 Range (min … max):  17.614 ms … 18.141 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     17.767 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.771 ms ± 66.985 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

               ▂▂▁▃▂ █▁▄▃█▃▄▄▂▆▁▂                              
  ▄▁▁▃▁▁▁▆▄▄▅▄▇█████▅████████████▇▇▄▅▅▆▅▁▃▃▁▁▃▄▁▄▁▁▁▃▁▁▁▁▁▁▁▃ ▄
  17.6 ms         Histogram: frequency by time          18 ms <

 Memory estimate: 382.39 KiB, allocs estimate: 5094.

88.04400243401969